<a href="https://colab.research.google.com/github/GGMmattos/TCC/blob/main/TCC_Gabriel_Matos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Libs

In [ ]:
import pandas as pd
import numpy as np
import torch
import re
import time
from transformers import AutoTokenizer, AutoModel
from google.colab import files
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split


!pip install pytorch-lightning -q
!pip install coral_pytorch -q

import torch
import os
import json # Para lidar com os embeddings salvos como string JSON
import ast # Para avaliar strings de lista de notas
import shlex # Para o NILC Metrix (se ainda precisar rodar o script no mesmo arquivo)

# Para PyTorch Lightning
import pytorch_lightning as pl
import torchmetrics
from coral_pytorch.layers import CoralLayer
from coral_pytorch.losses import coral_loss
from coral_pytorch.dataset import levels_from_labelbatch, proba_to_label

# Para Sklearn (pré-processamento e divisão de dados)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from torch.utils.data import Dataset, DataLoader

from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import CSVLogger


# Importação do dataset

**2018-Fonseca et al**

* Automatically Grading Brazilian Student Essays - NILC Metrix  - [link NILC](http://www.nilc.icmc.usp.br/nilc/projects/unitex-pb/web/dicionarios.html) -  https://github.com/nilc-nlp/nilcmetrix



In [ ]:
# Install huggingface_hub to inspect the repository
# !pip install huggingface_hub -q
# !huggingface-cli login
# import pandas as pd
# import numpy as np
# import torch
# from transformers import AutoTokenizer  # Or BertTokenizer
# from transformers import AutoModelForPreTraining  # Or BertForPreTraining for loading pretraining heads
# from transformers import AutoModel  # or BertModel, for BERT without pretraining heads

Importação do dataset [Link](https://huggingface.co/datasets/kamel-usp/aes_enem_dataset)

In [ ]:
# # Usado no trabalho de referência Silveira (2024)
# splits = {'train': 'PROPOR2024/train-00000-of-00001.parquet', 'validation': 'PROPOR2024/validation-00000-of-00001.parquet', 'test': 'PROPOR2024/test-00000-of-00001.parquet'}
# df_PROPOR2024 = pd.read_parquet("hf://datasets/kamel-usp/aes_enem_dataset/" + splits["train"])

In [ ]:
# Login using e.g. `huggingface-cli login` to access this dataset
splits = {'train': 'PROPOR2024/train-00000-of-00001.parquet', 'validation': 'PROPOR2024/validation-00000-of-00001.parquet', 'test': 'PROPOR2024/test-00000-of-00001.parquet'}

# Load each split separately
df_train = pd.read_parquet("hf://datasets/kamel-usp/aes_enem_dataset/" + splits["train"]) # Talves seja interessante mudarmos um pouco, estamo usando só essa repartição a princípio
df_val = pd.read_parquet("hf://datasets/kamel-usp/aes_enem_dataset/" + splits["validation"])
df_test = pd.read_parquet("hf://datasets/kamel-usp/aes_enem_dataset/" + splits["test"])

# Concatenate the dataframes, or process them separately
df = pd.concat([df_train, df_val, df_test], ignore_index=True)

# Gerando embedding


BERTimbau (como a maioria dos BERTs base) tem um limite de 512 tokens para a entrada. Redações do ENEM geralmente são mais longas.

**Código para obter embedding (truncamento)** OBS: não utilizado para o nosso trabalho



Simplesmente cortar a redação após 512 tokens. Simples, mas pode perder informações críticas.

In [ ]:
# def get_bert_embedding(text, tokenizer, model, device, max_length=512):
#     inputs = tokenizer(text, return_tensors='pt', truncation=True, max_length=max_length, padding='max_length')
#     inputs = {k: v.to(device) for k, v in inputs.items()}

#     with torch.no_grad(): # Desativa o cálculo de gradientes para inferência, economiza memória
#         outputs = model(**inputs)

#     # Retorna o embedding do token [CLS] da última camada
#     # outputs.last_hidden_state[0, 0, :] é o embedding do [CLS]
#     return outputs.last_hidden_state[0, 0, :].cpu().numpy()

**Código para obter embedding (segmentação)**

Dividir a redação em pedaços de 512 tokens, gerar um embedding para cada pedaço e depois combiná-los

In [ ]:
def get_bert_embedding_segmented(text, tokenizer, model, device, max_length=512, stride=256):
    """
    Gera um embedding BERT para um texto longo, utilizando segmentação e agregação.

    Args:
        text (str): O texto da redação.
        tokenizer: O tokenizer do modelo BERT.
        model: O modelo BERT (ex: BERTimbau).
        device (torch.device): 'cuda' ou 'cpu'.
        max_length (int): O tamanho máximo de cada segmento para o BERT (default: 512).
        stride (int): O tamanho da sobreposição entre os segmentos (default: 256).
                      Um stride menor significa mais sobreposição e possivelmente
                      maior captura de contexto, mas mais processamento.

    Returns:
        np.array: O embedding combinado (média) de todos os segmentos, ou NaNs se o texto for nulo.
    """
    if pd.isna(text) or not isinstance(text, str): # Lidar com possíveis valores NaN ou não-string
        return np.full((model.config.hidden_size,), np.nan) # Retorna um array de NaNs

    # Tokenizar o texto completo com o retorno de IDs (para controlar a segmentação)
    # add_special_tokens=False para não adicionar [CLS]/[SEP] no começo e fim de cada segmento
    # pois queremos controlar isso.
    token_ids = tokenizer.encode(text, add_special_tokens=False)

    # Lista para armazenar os embeddings de cada segmento
    segment_embeddings = []

    # Iterar sobre os tokens para criar segmentos
    # max_length - 2 para dar espaço para [CLS] e [SEP] que serão adicionados pelo tokenizer em cada segmento
    effective_max_length = max_length - 2

    # Se o texto for muito curto, trata como um único segmento (mesmo que menor que max_length)
    if len(token_ids) <= effective_max_length:
        segments = [token_ids]
    else:
        segments = []
        for i in range(0, len(token_ids), effective_max_length - stride):
            segment = token_ids[i : i + effective_max_length]
            segments.append(segment)
            # Se o último segmento for menor que o stride (não suficiente para sobreposição),
            # ou se já chegamos ao final, paramos.
            if i + effective_max_length >= len(token_ids):
                break


    # Processar cada segmento
    for segment in segments:
        # Adicionar os tokens especiais para cada segmento individualmente
        # return_tensors='pt' para PyTorch
        # truncation=True é redundante aqui se já controlamos o tamanho, mas é bom manter
        # padding='max_length' garante que todos os segmentos tenham o mesmo tamanho de input
        inputs = tokenizer.prepare_for_model(
            segment,
            add_special_tokens=True,
            return_tensors='pt',
            max_length=max_length,
            padding='max_length',
            truncation=True # Em caso de erro na lógica de segmentação, isso garante
        )

        # Add batch dimension to each tensor in the inputs dictionary
        inputs = {k: v.unsqueeze(0).to(device) for k, v in inputs.items()}

        with torch.no_grad():
            outputs = model(**inputs)

        # Retorna o embedding do token [CLS] do segmento
        # outputs.last_hidden_state will now have shape (batch_size, sequence_length, hidden_size)
        # We still want the CLS token for the single item in the batch, which is index 0
        segment_embedding = outputs.last_hidden_state[0, 0, :].cpu().numpy()
        segment_embeddings.append(segment_embedding)

    if not segment_embeddings: # Se por algum motivo nenhum embedding foi gerado (ex: texto vazio após pré-processamento)
        return np.full((model.config.hidden_size,), np.nan)

    # Combinar os embeddings dos segmentos pela média
    combined_embedding = np.mean(segment_embeddings, axis=0)

    return combined_embedding

In [ ]:
# # 1. Carregar o Tokenizer e o Modelo BERTimbau
# tokenizer = AutoTokenizer.from_pretrained("neuralmind/bert-base-portuguese-cased")
# model = AutoModel.from_pretrained("neuralmind/bert-base-portuguese-cased")

# # Mover o modelo para a GPU, se disponível
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# model.to(device)
# model.eval() # Coloca o modelo em modo de avaliação

In [ ]:
# df['bert_embedding'] = df['essay_text'].apply(
#     lambda text: get_bert_embedding_segmented(text, tokenizer, model, device, max_length=512, stride=256)
# )

# print("\nDataFrame com embeddings BERTimbau segmentados:")
# print(df.head())

# # Para verificar o formato do primeiro embedding:
# print(f"\nShape do primeiro embedding segmentado: {df['bert_embedding'].iloc[0].shape}")

In [ ]:
# df.head()

In [ ]:
# nome_do_arquivo_csv = 'DataFrame_Final.csv'
# df.to_csv(nome_do_arquivo_csv, index=False)

# Ajuste no dataframe e Divisão (Treino, Validação, Teste)

In [ ]:
# df = pd.read_csv('/content/drive/MyDrive/TCC -  Bacharel  Informática/Arquivos desenvolvimento/Base de dados/DataFrame_Final/DataFrame_Final-2.0.csv')

In [ ]:
# Criação da matrix X de features (contém as métricas e os embedding das redações)
# X = df.loc[:, 'adjective_ratio':'ratio_function_to_content_words']

In [ ]:
# print("Tipos de dados ANTES da limpeza:")
# print(X.dtypes.head(10)) # Mostra as primeiras colunas
# print("\n---")

In [ ]:
# colunas_para_limpar = [col for col in X.columns] # pegamos todas as métricas

In [ ]:
# # Iterar sobre as colunas e limpar/converter
# for col in colunas_para_limpar:
#     # APLICAR A LIMPEZA SOMENTE SE A COLUNA FOR DE TIPO 'object' (string)
#     if X[col].dtype == 'object':
#         # Substitui o ponto (separador de milhar) por vazio
#         # E substitui a vírgula (separador decimal, se houvesse) por ponto
#         # Depois, converte para numérico
#         # errors='coerce' converte qualquer valor que não possa ser numérico para NaN
#         X[col] = X[col].astype(str).str.replace('.', '', regex=False).str.replace(',', '.', regex=False)
#         X[col] = pd.to_numeric(X[col], errors='coerce') # Use pd.to_numeric para conversão robusta

# # Verificar os dtypes DEPOIS da limpeza
# print("Tipos de dados DEPOIS da limpeza:")
# print(X.dtypes.head(10))
# print("\n---")

Procedimento para não gerar erro ao tentar utilizar o **StandardScaler**

- Isso evita que features com valores maiores dominem o processo de aprendizado.

In [ ]:
# # Converter a string para array NumPy
# def string_to_array(embedding_str):
#     # Remove colchetes e espaços extras, depois divide pelos espaços
#     embedding_str = embedding_str.strip('[]')
#     # Divide pelos espaços e quebras de linha, filtrando valores vazios
#     numbers = [float(x) for x in embedding_str.replace('\n', ' ').split() if x]
#     return np.array(numbers)

# # Aplica a conversão para toda a coluna
# conversao_embedding = df['bert_embedding'].apply(string_to_array)

In [ ]:
# X_metrics  = np.vstack(X.values)
# X_embeddings = np.vstack(conversao_embedding.values)
# X_combined = np.hstack((X_metrics, X_embeddings))

In [ ]:
# scaler = StandardScaler()
# X_scaled = scaler.fit_transform(X_combined)

In [ ]:
# X_scaled

**Divisão das competências da classe alvo (y)**

In [ ]:
# y = df['grades'].values

In [ ]:
# y_split = np.array([np.fromstring(grade_str[1:-1], sep=' ') for grade_str in y])
# y_competencia1 = y_split[:, 0]  # Notas da competência 1
# y_competencia2 = y_split[:, 1]  # Notas da competência 2
# y_competencia3 = y_split[:, 2]  # Notas da competência 3
# y_competencia4 = y_split[:, 3]  # Notas da competência 4
# y_competencia5 = y_split[:, 4]  # Notas da competência 5
# y_nota_final = y_split[:, 5]    # Nota final (soma das competências)

**Config para a competência 1**

Divisão do Dataset (Treino, Validação, Teste)

In [ ]:
# # Primeiro divide em treino + temp_teste
# X_train0, X_temp0, y_train0, y_temp0 = train_test_split(X_scaled, y_split, test_size=0.3, random_state=42, stratify=y)

# # Depois divide temp_teste em validação e teste
# X_val0, X_test0, y_val0, y_test0 = train_test_split(X_temp0, y_temp0, test_size=0.5, random_state=42, stratify=y_temp)

# # O 'stratify=y' é MUITO importante para garantir que a distribuição das notas
# # seja semelhante em todos os conjuntos, o que é crucial para problemas ordinais.

O código acima resultou em um erro porque o **stratify=y** exige pelo menos 2 exemplos por classe para manter a proporção, e a saída mostra que muitas classes têm apenas 1 exemplo.

Algumas possíveis soluções?

* Agrupe classes raras (com 1 ou 2 exemplos) em uma categoria mais ampla (ex: "OUTROS").

* Remova o stratify (não recomendado, pois perde o balanceamento).

* Use oversampling (ex: SMOTE) para aumentar exemplos das classes raras.

Remoção do stratify

In [ ]:
# X_train1, X_temp1, y_train1, y_temp1 = train_test_split(X_scaled, y_competencia1, test_size=0.3, random_state=42)
# X_val1, X_test1, y_val1, y_test1 = train_test_split(X_temp1, y_temp1, test_size=0.5, random_state=42)

Resultado da Divisão (ao menos a princípio):
* Treino: 70% dos dados.
* Validação: 15% dos dados .
* Teste: 15% dos dados oriinais.

**Config para a competência 2**

In [ ]:
# X_train2, X_temp2, y_train2, y_temp2 = train_test_split(X_scaled, y_competencia2, test_size=0.3, random_state=42)
# X_val2, X_test2, y_val2, y_test2 = train_test_split(X_temp2, y_temp2, test_size=0.5, random_state=42)

**Config para a competência 3**

In [ ]:
# X_train3, X_temp3, y_train3, y_temp3 = train_test_split(X_scaled, y_competencia3, test_size=0.3, random_state=42)
# X_val3, X_test3, y_val3, y_test3 = train_test_split(X_temp3, y_temp3, test_size=0.5, random_state=42)

**Config para a competência 4**

In [ ]:
# X_train4, X_temp4, y_train4, y_temp4 = train_test_split(X_scaled, y_competencia4, test_size=0.3, random_state=42)
# X_val4, X_test4, y_val4, y_test4 = train_test_split(X_temp4, y_temp4, test_size=0.5, random_state=42)

**Config para a competência 5**

In [ ]:
# X_train5, X_temp5, y_train5, y_temp5 = train_test_split(X_scaled, y_competencia5, test_size=0.3, random_state=42)
# X_val5, X_test5, y_val5, y_test5 = train_test_split(X_temp5, y_temp5, test_size=0.5, random_state=42)

#  Regressão Ordinal usando CORAL

In [ ]:
# df = pd.read_csv('/content/drive/MyDrive/TCC -  Bacharel  Informática/Arquivos desenvolvimento/Base de dados/DataFrame_Final/DataFrame_Final-2.0.csv')

Processamento da coluna de notas


In [ ]:

# # 1. Converter as strings de array para listas/arrays NumPy reais
# print("Convertendo strings de notas (coluna 'grades') para listas de números...")
# df['grades_parsed'] = df['grades'].apply(
#     # 1. .strip('[]') remove os colchetes
#     # 2. .split() divide por qualquer espaço em branco (um ou vários)
#     # 3. int(s) converte cada parte para inteiro
#     lambda x: [int(s) for s in x.strip('[]').split()] if isinstance(x, str) else x
# )
# print(f"Exemplo da coluna 'grades_parsed':\n{df['grades_parsed'].head()}")


In [ ]:
# # 2. Extrair cada nota de competência e a nota total
# print("Extraindo notas para cada competência e a nota total...")
# # Nomes das colunas para as notas de competência
# competencia_cols = [f'nota_competencia_{i+1}' for i in range(5)]
# # Coluna para a nota total
# nota_total_col = 'nota_final_redacao'

# # Iterar para criar as colunas de competência e a nota final
# for i in range(6): # Iterar de 0 a 5 para pegar os 6 valores do vetor
#     col_name = ''
#     if i < 5:
#         col_name = competencia_cols[i]
#     else: # O sexto elemento (índice 5) é a nota total
#         col_name = nota_total_col

#     df[col_name] = df['grades_parsed'].apply(
#         # Verifica se é uma lista e tem o tamanho esperado (6 elementos)
#         lambda x: x[i] if isinstance(x, list) and len(x) == 6 else np.nan
#     )
#     # Garante que as notas sejam inteiras
#     df[col_name] = df[col_name].astype(int)

# print("Exemplo das novas colunas de notas de competência e total:")
# print(df[competencia_cols + [nota_total_col]].head())
# print("\n---")

In [ ]:
# # 4. Definir NUM_CLASSES para cada Competência e Mapeamento Ordinal
# NUM_CLASSES_COMPETENCIA = 6 # 0, 40, 80, 120, 160, 200

# # Mapeamento para o formato ordinal (0 a 5)
# notas_competencia_unicas = np.array([0, 40, 80, 120, 160, 200])
# mapeamento_competencia_ordinal = {nota: i for i, nota in enumerate(notas_competencia_unicas)}

# # Aplicar o mapeamento para cada coluna de competência
# print(f"Mapeando notas de competência para o formato ordinal (0 a {NUM_CLASSES_COMPETENCIA-1})")
# for col_name in competencia_cols:
#     df[f'{col_name}_ordinal'] = df[col_name].map(mapeamento_competencia_ordinal)

# print("Exemplo das notas ordinais das competências:")
# print(df[[f'{col_name}_ordinal' for col_name in competencia_cols]].head())

In [ ]:
# nome_do_arquivo_csv = 'DataFrame_Final-3.0.csv'
# df.to_csv(nome_do_arquivo_csv, index=False)

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/TCC -  Bacharel  Informática/Arquivos desenvolvimento/Base de dados/DataFrame_Final/DataFrame_Final-3.0.csv')

In [ ]:
df.head()

,id_texto,essay_text,grades,adjective_ratio,adverbs,content_words,flesch,function_words,sentences_per_paragraph,syllables_per_content_word,...,nota_competencia_2,nota_competencia_3,nota_competencia_4,nota_competencia_5,nota_final_redacao,nota_competencia_1_ordinal,nota_competencia_2_ordinal,nota_competencia_3_ordinal,nota_competencia_4_ordinal,nota_competencia_5_ordinal
0,redacao_1,"Ultimamente, temos observado, um aumento consi...",[0 0 0 0 0 0],5.495,5.495,54.396,1.201.548,45.604,1.500,272.727,...,0,0,0,0,0,0,0,0,0,0
1,redacao_2,"Infelizmente, no Brasil, na maioria dos lugare...",[0 0 0 0 0 0],7.107,3.553,58.883,3.488.631,41.117,2.000,291.379,...,0,0,0,0,0,0,0,0,0,0
2,redacao_3,"Em todos os lugares do Brasil temos violência,...",[0 0 0 0 0 0],2.564,5.128,53.846,4.271.548,46.154,133.333,279.365,...,0,0,0,0,0,0,0,0,0,0
3,redacao_4,"No Brasil, o número de cidadãos que querem mig...",[0 0 0 0 0 0],3.571,5.952,61.111,43.783,38.889,3.000,283.117,...,0,0,0,0,0,0,0,0,0,0
4,redacao_5,"Com apenas 20 anos de idade, o poeta romântico...",[ 200 200 200 200 200 1000],7.459,663.000,61.602,327.519,38.398,4.000,304.036,...,200,200,200,200,1000,5,5,5,5,5


Configurações gerais e hiperparâmetros


In [ ]:
BATCH_SIZE = 128
NUM_EPOCHS = 400
LEARNING_RATE = 0.01
NUM_WORKERS = 0 # Este parâmemtro pode ajudar na velocidade do treinamento não na qualidade do modelo em termos das métricas
# Notas de competência do ENEM: 0, 40, 80, 120, 160, 200
NUM_CLASSES_COMPETENCIA = 6

**MultiLayerPerceptron**: Esta é a rede neural em "PyTorch puro". Ela define a arquitetura do Perceptron Multicamadas e incorpora a camada de saída especial do CORAL (CoralLayer).

In [ ]:
class MultiLayerPerceptron(torch.nn.Module):
    def __init__(self, input_size, hidden_units, num_classes):
        super().__init__() # Chama o construtor da classe base torch.nn.Module

        # Armazena o número de classes, necessário para a função de perda CORAL
        self.num_classes = num_classes

        # Lista para armazenar todas as camadas da MLP
        all_layers = []

        # Loop para criar as camadas ocultas da MLP
        # 'hidden_units' é uma tupla ou lista (ex: (256, 128, 64))
        # Cada 'hidden_unit' representa o número de neurônios em uma camada oculta.
        for hidden_unit in hidden_units:
            # Cria uma camada linear (totalmente conectada)
            # 'input_size' é o número de entradas para esta camada (saída da camada anterior ou features iniciais)
            # 'hidden_unit' é o número de saídas desta camada
            layer = torch.nn.Linear(input_size, hidden_unit)
            all_layers.append(layer)

            # Adiciona uma função de ativação ReLU após cada camada linear (exceto a última do CORAL)
            all_layers.append(torch.nn.ReLU())

            # Atualiza o 'input_size' para a próxima camada ser a saída da camada atual
            input_size = hidden_unit

        # --- Camada de Saída CORAL ---
        # Esta é a principal adaptação para Regressão Ordinal com CORAL.
        # Ao invés de uma camada linear normal (torch.nn.Linear) que retornaria um vetor de logits para classificação multiclasse,
        # usamos a CoralLayer da biblioteca coral_pytorch.
        # `size_in`: número de entradas para esta camada (que é a saída da última camada oculta, hidden_units[-1])
        # `num_classes`: o número total de categorias de pontuação (ex: 6 para 0 a 200).
        output_layer = CoralLayer(size_in=hidden_units[-1], num_classes=num_classes)
        all_layers.append(output_layer)

        # Combina todas as camadas em um modelo sequencial.
        # 'Sequential' executa as camadas em ordem, uma após a outra.
        self.model = torch.nn.Sequential(*all_layers)

    def forward(self, x):
        """
        Define o fluxo de dados para frente através da rede neural.
        `x` é o tensor de entrada (as features).
        """
        # Passa o tensor de entrada através da sequência de camadas definidas
        x = self.model(x)
        return x

**LightningMLP**: Esta é uma classe do PyTorch Lightning. Ela "envolve" a rede neural (MultiLayerPerceptron) e adiciona toda a funcionalidade extra que o Lightning oferece (treinamento automatizado, validação, teste, logs, otimizadores, etc.), tornando o código de treino muito mais limpo e padronizado.

In [ ]:
class LightningMLP(pl.LightningModule):
    def __init__(self, model, learning_rate):
        super().__init__() # Chama o construtor da classe base pl.LightningModule

        self.learning_rate = learning_rate # Taxa de aprendizado para o otimizador
        self.model = model # A instância da rede neural MultiLayerPerceptron

        # Salva configurações e hiperparâmetros (como learning_rate) no diretório de log.
        # 'ignore=['model']' impede que os parâmetros do modelo sejam salvos duas vezes.
        self.save_hyperparameters(ignore=['model'])

        # --- Configuração das Métricas de Avaliação ---
        # pythorthmetrics são uma forma conveniente de calcular métricas.
        # Mean Absolute Error (MAE): Mede a diferença média absoluta entre previsões e rótulos verdadeiros.
        self.train_mae = torchmetrics.MeanAbsoluteError()
        self.valid_mae = torchmetrics.MeanAbsoluteError()
        self.test_mae = torchmetrics.MeanAbsoluteError()

        # Quadratic Weighted Kappa (QWK): Métrica CRUCIAL para AES.
        # Ela considera a natureza ordinal das notas e penaliza erros maiores mais severamente.
        # `num_classes`: número de categorias de nota (ex: 6 para competências).
        # `task='multiclass'`: indica que é uma tarefa de classificação multi-classe (embora seja ordinal).
        # `weights='quadratic'`: Aplica os pesos quadráticos para penalizar mais erros maiores.
        self.train_qwk = torchmetrics.CohenKappa(num_classes=self.model.num_classes, task='multiclass', weights='quadratic')
        self.valid_qwk = torchmetrics.CohenKappa(num_classes=self.model.num_classes, task='multiclass', weights='quadratic')
        self.test_qwk = torchmetrics.CohenKappa(num_classes=self.model.num_classes, task='multiclass', weights='quadratic')

    def forward(self, x):
        """
        Define o fluxo de dados para frente através do modelo PyTorch.
        Este método é chamado internamente pelo Lightning para fazer previsões.
        """
        return self.model(x) # Simplesmente passa a entrada para o MultiLayerPerceptron

    def _shared_step(self, batch):
        """
        Etapa comum de processamento que é usada para treinamento, validação e teste.
        Isso evita a duplicação de código.
        """
        features, true_labels = batch # Desempacota o batch de dados

        # --- Adaptação CORAL: Converter labels para o formato binário estendido ---
        # 'levels_from_labelbatch' é uma função de coral_pytorch.
        # Ela transforma um rótulo de classe inteira (ex: 3) em um vetor binário de "níveis" (ex: [1, 1, 1, 0, 0, 0])
        # `num_classes`: o número total de classes.
        levels = levels_from_labelbatch(
            true_labels, num_classes=self.model.num_classes)

        # Passa as features para a rede neural para obter os logits (saídas brutas)
        logits = self(features) # self(features) é o mesmo que self.forward(features)

        # --- Adaptação CORAL: Calcular a função de perda CORAL ---
        # 'coral_loss' é uma função de coral_pytorch.
        # Ela calcula a perda entre os logits do modelo e os 'levels' binários.
        # 'levels.type_as(logits)' garante que os tipos de dados dos tensores sejam compatíveis.
        loss = coral_loss(logits, levels.type_as(logits))

        # --- Adaptação CORAL: Converter probabilidades previstas em rótulos finais ---
        # `torch.sigmoid(logits)`: Transforma os logits brutos em probabilidades entre 0 e 1.
        # `proba_to_label`: Uma função de coral_pytorch que converte essas probabilidades
        # em rótulos de classe previstos (ex: 0, 1, 2, ..., num_classes-1).
        probas = torch.sigmoid(logits)
        predicted_labels = proba_to_label(probas)

        return loss, true_labels, predicted_labels

    def training_step(self, batch, batch_idx):
        """
        Define o que acontece em cada passo de treinamento (para cada batch).
        """
        loss, true_labels, predicted_labels = self._shared_step(batch) # Usa a etapa compartilhada

        # Registrar a perda de treinamento
        self.log("train_loss", loss, on_epoch=True, on_step=False) # 'on_epoch=True' loga no final da epoch

        # Calcular e registrar o MAE de treinamento
        self.train_mae(predicted_labels, true_labels)
        self.log("train_mae", self.train_mae, on_epoch=True, on_step=False)

        # Calcular e registrar o QWK de treinamento
        self.train_qwk(predicted_labels, true_labels)
        self.log("train_qwk", self.train_qwk, on_epoch=True, on_step=False)

        return loss  # A perda é retornada para o otimizador fazer o backpropagation

    def validation_step(self, batch, batch_idx):
        """
        Define o que acontece em cada passo de validação.
        Similar ao training_step, mas não calcula gradientes.
        """
        loss, true_labels, predicted_labels = self._shared_step(batch)

        self.log("valid_loss", loss, on_epoch=True, on_step=False)
        self.valid_mae(predicted_labels, true_labels)
        self.log("valid_mae", self.valid_mae,
                 on_epoch=True, on_step=False, prog_bar=True) # prog_bar mostra no progresso da barra
        self.valid_qwk(predicted_labels, true_labels)
        self.log("valid_qwk", self.valid_qwk,
                 on_epoch=True, on_step=False, prog_bar=True)

    def test_step(self, batch, batch_idx):
        """
        Define o que acontece em cada passo de teste (avaliação final).
        """
        # Não precisamos da perda para o teste, por isso o '_'
        _, true_labels, predicted_labels = self._shared_step(batch)

        self.test_mae(predicted_labels, true_labels)
        self.log("test_mae", self.test_mae, on_epoch=True, on_step=False)
        self.test_qwk(predicted_labels, true_labels)
        self.log("test_qwk", self.test_qwk, on_epoch=True, on_step=False)

    def configure_optimizers(self):
        """
        Configura o otimizador da rede neural.
        """
        # Otimizador Adam: otimizador popular e eficiente.
        # self.parameters() retorna todos os parâmetros treináveis do modelo.
        optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
        return optimizer

**Classe MyDataset**


Este código define a classe MyDataset, que prepara os dados para o PyTorch. Ele armazena as features (X) e labels (y) em arrays NumPy. Os métodos **__len__** e **__getitem__** permitem que o PyTorch saiba o tamanho total do seu dataset e como acessar cada amostra individualmente (features e seu label correspondente) usando um índice. Isso é fundamental para organizar os dados para o treinamento do modelo.


In [ ]:
class MyDataset(Dataset): # A classe MyDataset herda de torch.utils.data.Dataset
    def __init__(self, feature_array, label_array, dtype=np.float32):
        """
        Método construtor da classe. É chamado quando você cria uma nova instância de MyDataset.

        Args:
            feature_array (np.ndarray): Um array NumPy contendo suas features (X).
                                        Por exemplo, X_train_std, X_val_std, X_test_std.
            label_array (np.ndarray): Um array NumPy contendo seus labels (y).
                                      Por exemplo, y_train, y_val, y_test.
            dtype (np.float32, optional): O tipo de dado em que as features serão convertidas.
                                          np.float32 é um tipo comum para entradas de redes neurais,
                                          pois economiza memória e é compatível com GPUs.
        """
        # Converte o array de features para o tipo de dado especificado.
        # Isso é importante para garantir que as features estejam no formato numérico
        # esperado pelo PyTorch (geralmente float32 ou float64).
        self.features = feature_array.astype(dtype)

        # Armazena o array de labels.
        # Comentário: "Labels devem ser long para PyTorch" -- isso é uma dica importante.
        # Para problemas de classificação (e regressão ordinal como no presente trabalho, que usa classificadores binários internamente),
        # o PyTorch geralmente espera que os rótulos de classe sejam tensores do tipo Long (torch.long).
        # Se label_array for um NumPy array de inteiros, PyTorch lida com isso.
        self.labels = label_array

    def __getitem__(self, index):
        """
        Método mágico que permite acessar amostras do dataset usando índices, como em uma lista.
        Ex: dataset[0] chamaria __getitem__(0).

        Args:
            index (int): O índice da amostra que você deseja retornar.

        Returns:
            tuple: Uma tupla contendo (inputs, label) para a amostra no índice fornecido.
                   - inputs: As features (dados de entrada) da amostra.
                   - label: O rótulo (valor alvo) correspondente à amostra.
        """
        inputs = self.features[index] # Pega a linha de features no 'index'
        label = self.labels[index]   # Pega o label correspondente no 'index'
        return inputs, label

    def __len__(self, ):
        """
        Método mágico que retorna o número total de amostras no dataset.
        Permite usar len(dataset).

        Returns:
            int: O número de linhas (amostras) no array de features.
        """
        return self.features.shape[0] # Retorna o número de linhas (primeira dimensão) do array de features

**Classe CompetenceDataModule (Substitui DataModule)**


Esta é a parte central da adaptação para múltiplos modelos. O DataModule agora será específico para cada competência, recebendo os X e y já divididos e escalados.

In [ ]:
class CompetenceDataModule(pl.LightningDataModule): # A classe herda de pytorch_lightning.LightningDataModule
    def __init__(self, X_train, y_train, X_val, y_val, X_test, y_test, batch_size, num_workers):
        """
        Método construtor da classe. É chamado quando é criado uma nova instância de CompetenceDataModule.

        Args:
            X_train (np.ndarray): Matriz de features para o conjunto de treinamento.
            y_train (np.ndarray): Vetor de labels (notas) para o conjunto de treinamento.
            X_val (np.ndarray): Matriz de features para o conjunto de validação.
            y_val (np.ndarray): Vetor de labels para o conjunto de validação.
            X_test (np.ndarray): Matriz de features para o conjunto de teste.
            y_test (np.ndarray): Vetor de labels para o conjunto de teste.
            batch_size (int): O número de amostras por lote (batch) que o modelo processará de cada vez.
            num_workers (int): O número de subprocessos a serem usados para carregamento de dados.
                                0 significa que o carregamento será feito no processo principal.
                                Valores > 0 podem acelerar o carregamento, mas podem causar problemas em ambientes como o Windows/WSL.
        """
        super().__init__() # Chama o construtor da classe base pl.LightningDataModule

        # Armazena os arrays NumPy dos conjuntos de dados
        # Esses dados já devem estar pré-processados e escalados (ex: X_train_std)
        self.X_train = X_train
        self.y_train = y_train
        self.X_val = X_val
        self.y_val = y_val
        self.X_test = X_test
        self.y_test = y_test

        # Armazena configurações de DataLoader
        self.batch_size = batch_size
        self.num_workers = num_workers

    def setup(self, stage=None):
        """
        Este método é chamado pelo PyTorch Lightning para preparar os dados.
        Ele é chamado em cada "nó" (processo/GPU) em um ambiente distribuído.
        Normalmente, é aqui que fazemos a divisão de dados, escalamento, etc.
        Em nosso caso caso, ja fizemos isso no fluxo principal do script, então aqui ele
        apenas cria as instâncias de MyDataset.

        Args:
            stage (str, optional): Indica a fase atual ('fit', 'validate', 'test', 'predict').
                                   Permite lógica condicional se necessário. Ignorado aqui.
        """
        # Cria os objetos MyDataset para cada conjunto (treino, validação, teste).
        # MyDataset encapsula os arrays NumPy X e y em um formato que PyTorch pode usar.
        self.train = MyDataset(self.X_train, self.y_train)
        self.valid = MyDataset(self.X_val, self.y_val)
        self.test = MyDataset(self.X_test, self.y_test)

    def train_dataloader(self):
        """
        Retorna o DataLoader para o conjunto de treinamento.
        """
        return DataLoader(self.train,
                          batch_size=self.batch_size, # Tamanho dos lotes para o treino
                          num_workers=self.num_workers, # Número de subprocessos para carregar dados
                          shuffle=True, # Embaralha os dados a cada epoch para evitar que o modelo "decore" a ordem
                          drop_last=True) # Se o último batch não tiver o tamanho completo, ele é descartado.
                                          # Útil para modelos que esperam batches de tamanho fixo, ou GPUs.

    def val_dataloader(self):
        """
        Retorna o DataLoader para o conjunto de validação.
        """
        return DataLoader(self.valid,
                          batch_size=self.batch_size,
                          num_workers=self.num_workers)
                          # shuffle=False é o padrão para validação/teste, pois a ordem não importa.

    def test_dataloader(self):
        """
        Retorna o DataLoader para o conjunto de teste.
        """
        return DataLoader(self.test,
                          batch_size=self.batch_size,
                          num_workers=self.num_workers)
                          # shuffle=False é o padrão para validação/teste.

In [ ]:
# --- FLUXO PRINCIPAL: Carregar, Pré-processar e Treinar Múltiplos Modelos ---

#  Preparação da Matriz de Features (X)
# Converta a coluna 'bert_embedding' de string para array NumPy
print("\nPreparando a matriz de features (X)...", flush=True)

df['bert_embedding'] = df['bert_embedding'].apply(
    lambda x: (
        np.array(ast.literal_eval(
            # Passos mais robustos:
            # 1. str(x).strip('[]'): Converte para string, remove colchetes e espaços extras.
            # 2. re.sub(r'\s+', ',', ...): Substitui QUALQUER sequência de espaços (um ou mais) por UMA VÍRGULA.
            # 3. .replace(',,', ',') (Opcional): Lida com casos residuais de vírgulas duplas se o regex não for perfeito.
            # 4. .strip(',') (Opcional): Remove vírgulas extras no início/fim.
            re.sub(r'\s+', ',', str(x).strip('[]')).strip(',')
        ))
        if pd.notna(x) and isinstance(x, str) and x.strip() else np.nan
    )
)

# Depois de garantir que a coluna contém NumPy arrays ou NaNs, você pode empilhar:
bert_embeddings_matrix = np.stack(df['bert_embedding'].values)

# Limpar e converter colunas de métricas hand-crafted
primeira_coluna_metrica = 'adjective_ratio' # Verifique o nome da primeira métrica real
ultima_coluna_metrica = 'ratio_function_to_content_words' # Verifique o nome da última métrica real
colunas_handcrafted_nomes = df.loc[:, primeira_coluna_metrica:ultima_coluna_metrica].columns.tolist()


for col in colunas_handcrafted_nomes:
    if col in df.columns and df[col].dtype == 'object':
        df[col] = df[col].astype(str).str.replace('.', '', regex=False).str.replace(',', '.', regex=False)
        df[col] = pd.to_numeric(df[col], errors='coerce')

# Combinar as features
handcrafted_features_matrix = df[colunas_handcrafted_nomes].values
X_combined = np.hstack((bert_embeddings_matrix, handcrafted_features_matrix))
print("Matriz de features X_combined pronta.", flush=True)

In [ ]:
X_combined

In [ ]:
# --- 4. Loop para Treinar um Modelo para Cada Competência ---
resultados_por_competencia = {}
device_type = "cuda" if torch.cuda.is_available() else "cpu" #se GPU disponível usamos para o treinamento

for comp_idx in range(1, 6): # Para competência 1 a 5
    print(f"\n--- Treinando Modelo para Competência {comp_idx} ---", flush=True)

    # Define o vetor alvo (y) para a competência atual
    y_competencia = df[f'nota_competencia_{comp_idx}_ordinal'].values

    # Divisão do Dataset (X_combined e y_competencia)
    # stratify=y_competencia é CRUCIAL para dados desbalanceados
    X_train, X_temp, y_train, y_temp = train_test_split(
        X_combined, y_competencia, test_size=0.3, random_state=42, stratify=y_competencia
    )
    X_val, X_test, y_val, y_test = train_test_split(
        X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp
    )
    print(f"Divisão de dados para Competência {comp_idx}: Treino={X_train.shape}, Validação={X_val.shape}, Teste={X_test.shape}", flush=True)

    # Escalamento de Features (feito por um novo scaler para cada modelo/divisão)
    # Fit no treino, transform nos outros
    scaler_comp = StandardScaler()
    X_train_std = scaler_comp.fit_transform(X_train)
    X_val_std = scaler_comp.transform(X_val)
    X_test_std = scaler_comp.transform(X_test)
    print(f"Features escaladas para Competência {comp_idx}.", flush=True)

    # Opcional: Aplicar SMOTE APENAS no conjunto de treino se o desbalanceamento for severo
    # from imblearn.over_sampling import SMOTE
    # print(f"Aplicando SMOTE para Competência {comp_idx}...", flush=True)
    # smote = SMOTE(random_state=42)
    # X_train_resampled, y_train_resampled = smote.fit_resample(X_train_std, y_train)
    # X_train_std, y_train = X_train_resampled, y_train_resampled
    # print(f"SMOTE aplicado. Novo shape de treino: {X_train_std.shape}", flush=True)


    # Criar DataModule para a competência atual
    data_module_competencia = CompetenceDataModule(X_train_std, y_train, X_val_std, y_val, X_test_std, y_test, BATCH_SIZE, NUM_WORKERS)
    data_module_competencia.setup()

    # Inicializar e treinar o modelo PyTorch Lightning
    input_dim = X_combined.shape[1] # A dimensão de entrada é a mesma para todos os modelos

    pytorch_model_competencia = MultiLayerPerceptron(
        input_size=input_dim,
        hidden_units=(256, 128, 64), # Ajuste a arquitetura do MLP conforme testes
        num_classes=NUM_CLASSES_COMPETENCIA # Passa o NUM_CLASSES_COMPETENCIA = 6
    )
    lightning_model_competencia = LightningMLP(
        model=pytorch_model_competencia,
        learning_rate=LEARNING_RATE
    )

    # Callbacks e Logger
    # Salvar modelos e logs em pastas separadas para cada competência
    model_checkpoint_callback = ModelCheckpoint(
        save_top_k=1, mode="min", monitor="valid_mae",
        dirpath=f"logs_comp_{comp_idx}/", filename=f"best_model_comp_{comp_idx}"
    )
    logger_competencia = CSVLogger(save_dir="logs/", name=f"mlp-coral-comp-{comp_idx}")


    early_stop_callback = EarlyStopping(
    monitor="valid_qwk",  # Monitorar o QWK na validação
    min_delta=0.00,        # Mínima mudança para ser considerada uma melhoria, neste caso, consideramos qualquer melhora
    patience=50,           # Parar se o QWK não melhorar por 10 épocas
    verbose=True,
    mode="max"             # O "valid_qwk" deve ser maximizado
    )

    # Combine callbacks into a single list
    callbacks_list = [
        model_checkpoint_callback,
        early_stop_callback
    ]

    # Trainer
    trainer_competencia = pl.Trainer(
        max_epochs=NUM_EPOCHS,
        callbacks=callbacks_list,
        accelerator=device_type,
        devices=1,
        logger=logger_competencia,
        deterministic=True,
        log_every_n_steps=10
    )

    # Treinar
    start_time = time.time()
    print(f"Treinando modelo para Competência {comp_idx}...", flush=True)
    trainer_competencia.fit(model=lightning_model_competencia, datamodule=data_module_competencia)
    runtime = (time.time() - start_time)/60
    print(f"Treinamento para Competência {comp_idx} levou {runtime:.2f} min.", flush=True)

    # Avaliar no conjunto de teste (avaliação final)
    print(f"Avaliando modelo para Competência {comp_idx} no conjunto de teste...", flush=True)
    test_results = trainer_competencia.test(model=lightning_model_competencia, datamodule=data_module_competencia)
    resultados_por_competencia[f'competencia_{comp_idx}'] = test_results[0] # Armazenar os resultados do teste

    print(f"--- Modelo para Competência {comp_idx} concluído ---", flush=True)

print("\n--- Todos os modelos de competência foram treinados e avaliados ---", flush=True)
print("Resultados de Teste por Competência (MAE e QWK):", flush=True)
for comp, results in resultados_por_competencia.items():
    print(f"{comp}: Test MAE = {results['test_mae']:.4f}, Test QWK = {results['test_qwk']:.4f}", flush=True)